## Fun with the Genius API

Many web sites and organizations offer web APIs. We're going to go over how one API in particular works, or at least a subset of a particular web API---the [Genius API](https://docs.genius.com/). The idea is that by introducing you to this one API, you'll learn the tools necessary to sign up for, query, and interpret APIs from other providers as well.

### Signing up for a Client Access Token 

Before you can use the Genius API, you need to sign up for what they call a "client access token," which is another name for an API key. Do so by filling out the [New API Client form](https://genius.com/api-clients/new). If you don't yet have an account on Genius.com, you'll be prompted to register first. 

The next questions don't really apply to our use in class, but they're required to get your token. You can enter any words you want in "App Name." Similarly, you can enter any URL in the "Icon URL," "App Website URL," and "Redirect URI" fields.

Then, you'll be able to click on the "Generate Access Token" link to generate your client access token. 

The token is just a string of letters and numbers. It'll look something like this:

    6617c28c371f0a138f7912a35365564afe538605
    
That's your "key" for that API. Whenever you make a request to that API, you'll need to include your key in the request. The exact methodology for including the key will be explained below. (Note: the key above is just something I made up; it's not a valid key; don't try using it in actual requests.)

### Looking at the documentation

The documentation for a web API usually describes what the API can do in terms of the URLs you can access, the patterns that those URLs follow, and the format of the responses. If you click on "[Resources->Search](https://docs.genius.com/#search-h2)" on the documentation page, you see instructions for using the "search" function of the Genius API, for example.

This is a little confusing when you first look at it, but it's telling you the following:

* The "GET /search" line tells you that you should append "/search" to your URL path to make this request.
* The information in the blush-gray box tells you the parameter options. There is only one for search: "q" (that notation is pretty standard), and it can accept any search term as a value.

That's the only one we're going to use for now, so let's make a quick test request

In [6]:
import requests # requests again

base_url = "http://api.genius.com" # this is the URL for the Genius API
#Key line below here when, this is how to authorize your request when using the API
headers = {'Authorization': 'Bearer TOKEN'} ## REPLACE TOKEN W/ ACTUAL TOKEN <-- IMPORTANT !!!
search_url = base_url + "/search" # remember, this is how you format the URL for a search, as described above
song_title = "Respect" # We'll search for the first song title in our candidate playlist: Aretha Franklin's Respect 
params = {'q': song_title} # here's how you format the search parameter (see point two, above)

# and here's the API call
resp = requests.get(search_url, params=params, headers=headers)
data = resp.json()
data

{'meta': {'status': 200},
 'response': {'hits': [{'highlights': [],
    'index': 'song',
    'type': 'song',
    'result': {'annotation_count': 35,
     'api_path': '/songs/3042955',
     'full_title': 'N. J Respect R by\xa0Damso',
     'header_image_thumbnail_url': 'https://images.genius.com/14ca128d4ebb8fba259428db966b198e.300x169x1.jpg',
     'header_image_url': 'https://images.genius.com/14ca128d4ebb8fba259428db966b198e.1000x563x1.jpg',
     'id': 3042955,
     'lyrics_owner_id': 4012897,
     'lyrics_state': 'complete',
     'path': '/Damso-n-j-respect-r-lyrics',
     'pyongs_count': 24,
     'song_art_image_thumbnail_url': 'https://images.genius.com/1a572edbf6da198c99dacd7c19282804.300x300x1.jpg',
     'song_art_image_url': 'https://images.genius.com/1a572edbf6da198c99dacd7c19282804.1000x1000x1.jpg',
     'stats': {'unreviewed_annotations': 0, 'hot': False, 'pageviews': 459925},
     'title': 'N. J Respect R',
     'title_with_featured': 'N. J Respect R',
     'url': 'https://gen

This request is finding all songs that include the search string `Respect`. As described in the [documentation](https://docs.genius.com/#/authentication-h1), the results take the form of a dictionary with two keys: `response` (which points to a ditionary of a list of dictionaries; phew!) and `meta`, whose value is a string (`'status'`), which gives you the HTML status code for the response (i.e. whether the request was successful). 

Let's take a look at the top level:

In [3]:
data.keys()

dict_keys(['meta', 'response'])

From the output above, we already know that the response was successful. So we can ignore whatever is associated with the `meta` key. But let's dig a little deeper into the `response` key.

In [4]:
data['response'].keys()

dict_keys(['hits'])

So there is only one key, `hits`, which I will tell you comtains a list of dictionaries: one for each of the hits in the search result.

In [5]:
data['response']['hits'][0]

{'highlights': [],
 'index': 'song',
 'type': 'song',
 'result': {'annotation_count': 35,
  'api_path': '/songs/3042955',
  'full_title': 'N. J Respect R by\xa0Damso',
  'header_image_thumbnail_url': 'https://images.genius.com/14ca128d4ebb8fba259428db966b198e.300x169x1.jpg',
  'header_image_url': 'https://images.genius.com/14ca128d4ebb8fba259428db966b198e.1000x563x1.jpg',
  'id': 3042955,
  'lyrics_owner_id': 4012897,
  'lyrics_state': 'complete',
  'path': '/Damso-n-j-respect-r-lyrics',
  'pyongs_count': 24,
  'song_art_image_thumbnail_url': 'https://images.genius.com/1a572edbf6da198c99dacd7c19282804.300x300x1.jpg',
  'song_art_image_url': 'https://images.genius.com/1a572edbf6da198c99dacd7c19282804.1000x1000x1.jpg',
  'stats': {'unreviewed_annotations': 0, 'hot': False, 'pageviews': 459765},
  'title': 'N. J Respect R',
  'title_with_featured': 'N. J Respect R',
  'url': 'https://genius.com/Damso-n-j-respect-r-lyrics',
  'primary_artist': {'api_path': '/artists/45855',
   'header_imag

So this is what we want: the dictionary for each of the search results.

But lo and behold, it contains additional levels of data. 

Of these, we're most interested in the `result` dictionary. 

Important items in this dictionary are the song title itself (`title`), the url for the song lyrucs (`url`) and the `primary artist` key, which points to *another* dictionary with the name of the artist (`name`). 

The artist name could be used with a different API endpoint to get more detail about a particular artist. But this information is enough for our purposes today.

To get a sense of what we're looking at, let's get the full song title for each search hit:

In [6]:
# Remember list comprehension format: [ predicate expression FOR temporary variable name IN source list ]

[song['result']['title'] for song in data['response']['hits']]

['N. J Respect R',
 'Respect My Cryppin’',
 'Respect',
 'Respect the Game',
 'No Respect Freestyle',
 'Respect',
 'All Due Respect',
 'Money, Power, Respect',
 'Respect',
 'The Cypher Respect 2']

**Exercise:** Using the syntax above, list the URLs for the lyrics of each of these songs

In [2]:
[song['result']['url'] for song in data['response']['hits']]

['https://genius.com/Damso-n-j-respect-r-lyrics',
 'https://genius.com/Blueface-respect-my-cryppin-lyrics',
 'https://genius.com/Aretha-franklin-respect-lyrics',
 'https://genius.com/The-notorious-big-respect-lyrics',
 'https://genius.com/Meek-mill-respect-the-game-lyrics',
 'https://genius.com/Lil-peep-no-respect-freestyle-lyrics',
 'https://genius.com/Run-the-jewels-all-due-respect-lyrics',
 'https://genius.com/Travis-scott-money-power-respect-lyrics',
 'https://genius.com/Wiz-khalifa-respect-lyrics',
 'https://genius.com/Atentado-napalm-the-cypher-respect-2-lyrics']

**Exercise:** Adapting the syntax above, list the name of the artist for each of these songs.
    
Hint: Remember that `name` is contained with the dictionary `primary artist`

In [3]:
[song['result']['primary_artist']['name'] for song in data['response']['hits']]

['Damso',
 'Blueface',
 'Aretha Franklin',
 'The Notorious B.I.G.',
 'Meek Mill',
 'Lil Peep',
 'Run The Jewels',
 'Travis Scott',
 'Wiz Khalifa',
 'Atentado Napalm']

### An aside about query strings

So, one quick thing: what's with that `params` thing in the call to `requests.get()`? Well, it's there to make our lives easier. If you put `params=` into the arguments of the call to `requests.get()`, the library will take the dictionary that you passed and use it to form a query string. It does this so you don't have to figure out how to write the query string yourself.

Let's review what a query string looks like. Here's the example query string from the section above about URL structure:

    ?arg1=baz&arg2=quux
    
At first glance, it looks like garbage. For another perspective, let's look at the query string from the example request to the Genius API:

    ?token=6617c28c371f0a138f7912a35365564afe538605&q=Respect
    
A little bit of the structure becomes more apparent here! It looks like we have a series of key/value, separated by ampersands (`&`): `token=6617c28c371f0a138f7912a35365564afe538605` and `q=Respect`. The pairs themselves are separated by equal signs (`=`).

What we'd like to have, then, is some kind of way to write an expression that turns a Python dictionary into a string formatted correctly to include in a query string. It turns out that the process of doing this is [kind of tricky](http://en.wikipedia.org/wiki/Percent-encoding), so the `requests` library does the hard work for us when we pass the parameters to the `requests.get()` function with the named `params` parameter. Very handy!

### Working with responses

Now we've gotten a response from the API, and we've parsed it into a Python data structure that we know how to use (a dictionary). But now what do we do with it?

Well, let's find the URL for the lyrics for Aretha Franklin's "Respect," of course!

Remember that we've already got song_title stored from way back up at the top: it's what we searched for:

In [7]:
song_title

'Respect'

In [9]:
artist="Aretha Franklin" # you should already be thinking: how can I hook this up with the NYT article data...

lyrics_url = "" # create an empty string to hold the url

for song in data['response']['hits']: # loop through hits
    if song['result']['primary_artist']['name'] == artist: # check for match w/ artist
        lyrics_url = song['result']['url']  # if it matches, save the lyrics URL

lyrics_url

'https://genius.com/Aretha-franklin-respect-lyrics'

We've got our URL!

**If time:**

Let's write some pseudocode that links together the output of the NYT web scraping exercise with the Genius API to get lyrics URLS for each of the songs in the candidate playlist. 
    